In [230]:
#import sys

In [237]:
#sys.version
"""
작동전 유의사항

*** selenium 특성 상 작업 중 원격제어 중인 크롬창을 수동조작 시 오류가 발생할 가능성이 있으니, 코드 동작 시간동안은 가급적 조작하지 말 것.

0.pip or conda를 통해 패키지 설치
    0-1) 패키지 설치 방법은 구글링으로 알아볼 것

1.chrome driver 설치
    1-1) 옵션 > 도움말 > chrome 정보 > chrome 버전 확인
    1-2) https://chromedriver.chromium.org/downloads 해당 링크 접속 후 버전에 맞는 드라이버 설치
    1-3) 편의를 위해 설치한 크롬 드라이버는 현재 SARAM IN.ipynb 저장 경로와 동일한 곳에 이동 
         (경로가 다를 시 드라이버 위치를 직접 지정해야함)

2.SARAM_IN.xlsx의 경로를 수정
    2-1) 크롤링 결과가 입력되는 SARAM_IN.xlsx 파일은 최초 실행 전에 직접 파일을 만들어줘야함
    2-2) 빈 파일 생성 후 해당 파일의 경로 및 파일이름에 맞게 수정필요
    
3.결과 메일 발신은 개인 웹메일을 사용하므로, 개인 메일 계정 정보에 맞게 수정 필요
    3-1) https://yeolco.tistory.com/93 해당 링크 접속 후 중간 부분에 개인 웹메일 계정 및 비밀번호 발급 절차 시행
    3-2) 신규 발급한 계정 정보를 코드에 업데이트
    3-3) ******************* 해당 계정 정보는 개인의 gmail 로그인 정보로 코드 공유 시 pw는 꼭 지울 것! *******************
    
4.아래 코드 본문에 CODE_MAP에 저장되어 있는 희망 근무지역에 대한 정보들은 아래 코드에 내용이 없는 경우, 직접 사람인 사이트에 접속하여 
  페이지 소스에서 확인하고 동일 형식으로 추가해야함

5.메모리 사양이 부족해 창 전환이 느린 경우, 중간 중간에 time.sleep() 안에 숫자를 조절 (초단위)

6.이외에도 ctrl+f 로 '수정필요'를 검색하여, 해당 부분을 위 내용을 참고하여 꼭 수정할 것 (안 그러면 오류남)
    
"""


'\n작동전 유의사항\n0.pip or conda를 통해 패키지 설치\n    0-1) 패키지 설치 방법은 구글링으로 알아볼 것\n\n1.chrome driver 설치\n    1-1) 옵션 > 도움말 > chrome 정보 > chrome 버전 확인\n    1-2) https://chromedriver.chromium.org/downloads 해당 링크 접속 후 버전에 맞는 드라이버 설치\n    1-3) 편의를 위해 설치한 크롬 드라이버는 현재 SARAM IN.ipynb 저장 경로와 동일한 곳에 이동 (경로가 다를 시 드라이버 위치를 직접 지정해야함)\n\n2.SARAM_IN.xlsx의 경로를 수정\n    2-1) 크롤링 결과가 입력되는 SARAM_IN.xlsx 파일은 최초 실행 전에 직접 파일을 만들어줘야함\n    2-2) 빈 파일 생성 후 해당 파일의 경로 및 파일이름에 맞게 수정필요\n    \n3.결과 메일 발신은 개인 웹메일을 사용하므로, 개인 메일 계정 정보에 맞게 수정 필요\n    3-1) https://yeolco.tistory.com/93 해당 링크 접속 후 중간 부분에 개인 웹메일 계정 및 비밀번호 발급 절차 시행\n    3-2) 신규 발급한 계정 정보를 코드에 업데이트\n    3-3) ******************* 해당 계정 정보는 개인의 gmail 로그인 정보로 코드 공유 시 pw는 꼭 지울 것! *******************\n    \n4.아래 코드 본문에 CODE_MAP에 저장되어 있는 희망 근무지역에 대한 정보들은 아래 코드에 내용이 없는 경우, 직접 사람인 사이트에 접속하여 페이지 소스   \n\n5.selenium 특성 상 작업 중 원격제어 중인 크롬창을 수동조작 시 오류가 발생할 가능성이 있으니, 코드 동작 시간동안은 가급적 조작하지 말 것.\n    \n'

In [232]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pymysql
import time
import datetime
import os

import smtplib, os, pickle  # smtplib: 메일 전송을 위한 패키지
from email import encoders  # 파일전송을 할 때 이미지나 문서 동영상 등의 파일을 문자열로 변환할 때 사용할 패키지
from email.mime.text import MIMEText   # 본문내용을 전송할 때 사용되는 모듈
from email.mime.multipart import MIMEMultipart   # 메시지를 보낼 때 메시지에 대한 모듈
from email.mime.base import MIMEBase     # 파일을 전송할 때 사용되는 모듈

import xlsxwriter as XW

In [1]:
def SARAMIN(KEYWORD_LST,GENDER,AGE,CTY_LST,ToADDR,PATH):         
    
    #수정필요
    #RESULT = XW.Workbook(r'C:\\Users\\ghks9209\\Desktop\\개인\\취준\\' + PATH + '\\SARAM_IN.xlsx')
    RESULT = XW.Workbook(r'C:\\Users\\Hwanhee\\Desktop\\취준\\' + PATH + '\\SARAM_IN.xlsx')
    
    if GENDER == 'F':
        GENDER = '여성'
    else :
        GENDER = '남성'
    
    for SEARCH in KEYWORD_LST:
        SHEET = RESULT.add_worksheet(SEARCH)
        
        url = "http://www.saramin.co.kr/zf_user/search?search_area=main&search_done=y&search_optional_item=n&searchType=default_mysearch&searchword={SEARCH}".format(SEARCH = SEARCH)
        driver = webdriver.Chrome("chromedriver.exe")
        driver.get(url)
        time.sleep(5)
        
        CITY_LIST = driver.find_element_by_css_selector(".area_section .btn_area.btn_open_layer")
        CITY_LIST.click()

        CITY_LIST_OPEN = driver.find_element_by_css_selector(".wrap_list_check .area_btn .btn_all_category")
        CITY_LIST_OPEN.click()
        
        #수정필요 : 아래 리스트에 없는 지역 추가 희망시 직접 사람인 사이트 접속 후 페이지소스에서 확인 후 '지역명 : 소속 시도코드|해당 시군구코드' 입력
        CODE_MAP = {'서울' : '101000|101000' , '경기' : '102000|102000' , '광주' : '103000|103000' , '대구' : '104000|104000' , 
                    '대전' : '105000|105000' , '부산' : '106000|106000' , '울산' : '107000|107000' , '인천' : '108000|108000' , 
                    '강원' : '109000|109000' , '경남' : '110000|110000' , '경북' : '111000|111000' , '전남' : '112000|112000' , 
                    '전북' : '113000|113000' , '충북' : '114000|114000' , '충남' : '115000|115000' , '제주' : '116000|116000' , 
                    '전국' : '117000|117000' , '세종' : '118000|118000' , '청주' : '114000|114110'}

        for CTY in CTY_LST:
            CTY_CD = CODE_MAP[CTY].split('|')
            CITY_OPEN = driver.find_element_by_css_selector(".option_content.area_section.on .wrap_depth_category #depth1_btn_" + str(CTY_CD[0]) + " .depth1_btn_" + str(CTY_CD[0]))
            CITY_OPEN.click()

            if CTY_CD[0] != CTY_CD[1]:
                CITY_target = driver.find_element_by_xpath('//label[@for="loc_cd_' + str(CTY_CD[1]) +'"]')
                CITY_target.click() 

        DETAIL_OPTIONS = driver.find_element_by_css_selector(".main_option.active .detail_section .btn_detail_option ")
        DETAIL_OPTIONS.click()

        AGE_OPTION_BOX = driver.find_element_by_css_selector("#sp_optional_wrapper #sp_age .option_list")
        AGE_OPTION_BOX.click()

        AGE_SELECT = driver.find_elements_by_css_selector(".option_list #age_sel .link_opt")
        AGE_SELECT[AGE-17].click()

        GENDER_SELECT = driver.find_elements_by_css_selector("#sp_optional_wrapper #sp_gender .option_list.sri_radio_wrap.small .lbl")
        if GENDER == '여성':
            GENDER_SELECT[1].click()
        else :
            GENDER_SELECT[0].click()

        GO_FIND = driver.find_element_by_css_selector("#sp_main_wrapper #sp_preview #search_btn")
        GO_FIND.click()


        AREA_OPTION = driver.find_element_by_css_selector("#sp_main_wrapper .main_option.active .tab_section .area_section .btn_area.btn_open_layer .input_placeholder.filter_selected ").text
        start_point =  AREA_OPTION.find(") ")
        AREA_OPTION_INDEX = AREA_OPTION[start_point+2:]

        weekly_now = datetime.datetime.now()
        weekly_month = datetime.datetime(weekly_now.year,weekly_now.month,1)

        print('[ SEARCHING OPTIONS ]','-'*60,'\n')
        print('검색어 : ', SEARCH)
        print('구직자 연령 : ', AGE,'세')
        print('구직자 성별 : ', GENDER)
        print('희망근무지역 : ',AREA_OPTION_INDEX,'\n')
        print('*'*15,weekly_now.year,'년 ',weekly_now.month,'월 ',weekly_now.isocalendar()[1]-weekly_month.isocalendar()[1]+1,'주차 구인광고 크롤링을 시작합니다.','*'*15)

        GATHERING(SEARCH,AGE,AREA_OPTION_INDEX,PATH,RESULT,SHEET,driver)
        driver.close()
    
    RESULT.close()
    SEND_MAIL(KEYWORD_LST,AGE,GENDER,AREA_OPTION_INDEX,ToADDR,PATH)

In [4]:
def GATHERING(SEARCH,AGE,AREA_OPTION_INDEX,PATH,RESULT,SHEET,driver):
    SARAM_URLS = driver.find_elements_by_css_selector("#recruit_info_list .content .item_recruit .area_job .job_tit a")
    SARAM_COP_NAMES = driver.find_elements_by_css_selector("#recruit_info_list .content .item_recruit .area_corp .corp_name a")
        
    SHEET.write(0,0, "회사명")
    SHEET.write(0,1, "공고명")
    SHEET.write(0,2, "D-DAY")
    SHEET.write(0,3, "근무지역")
    SHEET.write(0,4, "근무형태")
    SHEET.write(0,5, "경력")
    SHEET.write(0,6, "학력")
    SHEET.write(0,7, "급여")
    SHEET.write(0,8, "우대조건")
    SHEET.write(0,9, "복리후생")
    SHEET.write(0,10, "URL")
    
    for i in range(len(SARAM_URLS)):
        NAME = SARAM_COP_NAMES[i].get_attribute('text')
        TITLE = SARAM_URLS[i].get_attribute('text')
        
        SARAM_URLS[i].click()
        time.sleep(1)
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(2)
       
        try:
            D_DAY = driver.find_element_by_css_selector(".wrap_jv_cont .wrap_jv_header .btn_apply .dday").text
        except:
            D_DAY = "정보없음"
        
        MAIN_INFO = driver.find_elements_by_css_selector(".wrap_jv_cont .jv_cont.jv_summary .cont .col dl")
        
        AREA = "정보없음"
        EMPLOYMENT_TYPE = "정보없음"
        EMP = "정보없음"
        CAREER = "정보없음"
        EDU = "정보없음"
        PAY = "정보없음"
        TIME = "정보없음"
        PREFER_INDEX = "정보없음"
        
        for MAIN_INFO_TYPE in MAIN_INFO: 
            if MAIN_INFO_TYPE.text[0:4] == "근무지역":
                AREA = MAIN_INFO_TYPE.text.replace("근무지역","").replace("지도","").replace("\n","")
            elif MAIN_INFO_TYPE.text[0:4] == "근무형태": 
                EMP = MAIN_INFO_TYPE.text.replace("근무형태","").replace("\n","")
            elif MAIN_INFO_TYPE.text[0:2] == "경력": 
                CAREER = MAIN_INFO_TYPE.text.replace("경력","").replace("\n","")
            elif MAIN_INFO_TYPE.text[0:2] == "학력": 
                EDU = MAIN_INFO_TYPE.text.replace("학력","").replace("\n","")
            elif MAIN_INFO_TYPE.text[0:2] == "급여": 
                PAY = MAIN_INFO_TYPE.text.replace("급여","").replace("\n","")
            elif MAIN_INFO_TYPE.text[0:4] == "근무일시": 
                TIME = MAIN_INFO_TYPE.text.replace("근무일시","").replace("\n","")
            elif MAIN_INFO_TYPE.text[0:4] == "우대사항": 
                PREFER = driver.find_element_by_css_selector(".wrap_jv_cont .jv_cont.jv_summary .cont .col dl .preferred .toolTipWrap .spr_jview.btn_jview.btn_tooltip")
                PREFER.click()
                PREFER_INDEX_LIST = driver.find_elements_by_css_selector(".wrap_jv_cont .jv_cont.jv_summary .cont .col dl .preferred .toolTipWrap.opened .toolTip .toolTipCont.txtLeft .toolTipTxt li")
                PREFER_INDEX = ""
                for j in range(len(PREFER_INDEX_LIST)):
                    PREFER_INDEX += '  (' + str(j+1) + ') ' +PREFER_INDEX_LIST[j].text.replace("\n"," | ")+'\n'
                break
            else :
                PREFER_INDEX = " 정보없음" + '\n'
        
        BENEFIT_BANNER = []
        BENEFIT_BANNER = driver.find_elements_by_css_selector(".wrap_jview .wrap_jv_cont .jv_title")
        BENEFIT_INDEX = ""
        
        for y in range(len(BENEFIT_BANNER)):
            if BENEFIT_BANNER[y].text[0:4] == "복리후생":
                BENEFIT_LIST = driver.find_elements_by_css_selector(".cont .layer_group .col")
                BENEFIT_INDEX = ""
                for x in range(len(BENEFIT_LIST)):
                    if BENEFIT_LIST[x].text == "" :
                        break
                    elif BENEFIT_LIST[x].text[0:4] == "고졸이하":
                        BENEFIT_INDEX = " 정보없음"
                        break
                    elif BENEFIT_LIST[x].text[0:2] == "신입":
                        BENEFIT_INDEX = " 정보없음"
                        break
                    else:
                        BENEFIT_INDEX +='  (' + str(x+1) + ') ' + BENEFIT_LIST[x].text.replace("\n"," | ")+'\n'
                break
                
            else:
                BENEFIT_INDEX = " 정보없음"
            
        PAGE_URL = driver.current_url
        START_POINT =  PAGE_URL.find("link href=\"http://www.saramin.co.kr")
        END_POINT =  PAGE_URL.find("rel=\"canonical")
        PAGE_URL_INDEX = PAGE_URL
        
        SHEET.write(i+1,0, NAME)
        SHEET.write(i+1,1, TITLE)
        SHEET.write(i+1,2, D_DAY)
        SHEET.write(i+1,3, AREA)
        SHEET.write(i+1,4, EMP)
        SHEET.write(i+1,5, CAREER)
        SHEET.write(i+1,6, EDU)
        SHEET.write(i+1,7, PAY)
        #SHEET.write(i+1,8, TIME)
        SHEET.write(i+1,8, PREFER_INDEX)
        SHEET.write(i+1,9, BENEFIT_INDEX)
        SHEET.write(i+1,10, PAGE_URL_INDEX)
                        
        print('\n','[ NO.',i+1,']','\n')
        print('0)   URL    : ',PAGE_URL_INDEX)
        print('1)   회사명 : ',NAME)
        print('2)   공고명 : ',TITLE)
        print('3)    D-DAY : ',D_DAY)
        print('4) 근무지역 : ',AREA)
        print('5) 근무형태 : ',EMP)
        print('6)     경력 : ',CAREER)
        print('7)     학력 : ',EDU)
        print('8)     급여 : ',PAY)
        #print('9) 근무일시 : ',TIME)
        print('9) 우대조건 \n',PREFER_INDEX)                                                                 
        print('10) 복리후생 \n',BENEFIT_INDEX)                                                                 
                                                                         
              
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(1)
        
    #RESULT.close()

In [3]:
def SEND_MAIL(KEYWORD_LST,AGE,GENDER,AREA_OPTION_INDEX,ToADDR,PATH):
    
    #수정필요 : *************공유시 pw 부분 삭제 요망************
    E_MAIL = "ghks9209@gmail.com"
    
    SMTP = smtplib.SMTP('smtp.gmail.com', 587)   # 587: 서버의 포트번호
    SMTP.starttls()   # tls방식으로 접속, 그 포트번호가 587
    #수정필요
    SMTP.login('ghks9209@gmail.com',PW)
    
    for ADDR in ToADDR:
        weekly_now = datetime.datetime.now()
        weekly_month = datetime.datetime(weekly_now.year,weekly_now.month,1)
        MSG = MIMEBase('multipart','mixed')
        MSG_TEXT = MIMEText('*'*3 + str(weekly_now.year) + '년 ' + str(weekly_now.month) + '월 ' + str(weekly_now.isocalendar()[1]-weekly_month.isocalendar()[1]+1) + '주차 구인광고 크롤링 결과' + '\n 검색어 : ' + str(KEYWORD_LST) + ' \n 성별 : ' + GENDER + '(성별 무관 포함) \n 나이 : ' + str(AGE) + '세 \n 희망 근무지역 : ' + AREA_OPTION_INDEX)
        MSG['Subject'] = '사람인 구인공고 크롤링 결과'
        MSG_TEXT['Subject'] = '사람인 구인공고 크롤링 결과'
        #수정필요
        MSG_TEXT['FROM'] = 'ghks9209@gmail.com'
        MSG_TEXT['TO'] = ADDR
        
        MSG.attach(MSG_TEXT)
        #수정필요
        #path = r'C:\\Users\\ghks9209\\Desktop\\개인\\취준\\' + PATH + '\\SARAM_IN.xlsx'
        path = r'C:\\Users\\Hwanhee\\Desktop\\취준\\' + PATH + '\\SARAM_IN.xlsx'
        PART = MIMEBase("application","octet-stream")
        PART.set_payload(open(path,'rb').read())
        
        encoders.encode_base64(PART)
        PART.add_header('Content-Disposition',
                       'attachment;filename="%s"'% os.path.basename(path))
        MSG.attach(PART)
        
        #수정필요
        SMTP.sendmail("ghks9209@gmail.com", ADDR, MSG.as_string())
        print(ADDR,'  END')

In [238]:
#환희
#HH_KEYWORD = ['데이터분석가','데이터 사이언티스트','데이터컨설팅','빅데이터']
#HH_CTY_LST = ['서울','경기','대전']
#HK_E_ADDR = ['ghks9209@openmate.co.kr']
#SARAMIN(HH_KEYWORD,'M',30,HH_CTY_LST,HK_E_ADDR,'HH')


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=91.0.4472.101)


'//label[@for="loc_mcd_114110"]'